In [2]:
import pandas as pd
import pyreadr
import numpy as np
from tqdm import tqdm

from unidecode import unidecode

In [3]:
#5570 muns
#legal amazon has ~700 

#cpf variable to identify candidates

#variables: year, mun, winner of round 1, winner of round 2, total votes round, total votes round 2
            #margin for round1, margin for round 2, is incumbant running, 

In [4]:
path = '/Users/annieulichney/Library/CloudStorage/Box-Box/Deforestation_MachineLearning/Data/cand_prefeito.Rdata'
pathvote = '/Users/annieulichney/Library/CloudStorage/Box-Box/Deforestation_MachineLearning/Data/cand_prefeito_vote.Rdata'

df = pyreadr.read_r(path)['cand']
vote1 = pyreadr.read_r(pathvote)['cand']

vote1 = vote1[vote1.tipo_eleicao == 'eleicao ordinaria']

df = df[~df.situacao.isin(['deferido', 'deferido com recurso', 'sub judice', 'pendente de julgamento', 
                           'substituto majoritario pendente de julgamento', 'substituto pendente de julgamento'])]


In [5]:
vote1.head()

,ano,turno,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,cargo,sigla_partido,numero_candidato,sequencial_candidato,id_candidato_bd,resultado,votos
0,1996,1,eleicao ordinaria,AL,2704302,27855,prefeito,PSD,41,NaN,NaN,nao eleito,2388
1,1996,1,eleicao ordinaria,AM,1302603,2550,prefeito,PSTU,16,NaN,NaN,nao eleito,1713
2,1996,1,eleicao ordinaria,BA,2927408,38490,prefeito,PAN,26,NaN,NaN,nao eleito,15721
3,1996,1,eleicao ordinaria,BA,2927408,38490,prefeito,PCO,29,NaN,NaN,nao eleito,705
4,1996,1,eleicao ordinaria,BA,2927408,38490,prefeito,PGT,30,NaN,NaN,nao eleito,989


In [6]:
vote1.columns

Index(['ano', 'turno', 'tipo_eleicao', 'sigla_uf', 'id_municipio',
       'id_municipio_tse', 'cargo', 'sigla_partido', 'numero_candidato',
       'sequencial_candidato', 'id_candidato_bd', 'resultado', 'votos'],
      dtype='object')

In [7]:
vote_2016 = pd.read_csv('election_results_2016.csv')
vote_2016 = vote_2016[vote_2016.tipo_eleicao == 'eleicao ordinaria']
vote_2016.id_municipio = (vote_2016.id_municipio.astype(int, errors = 'ignore')).astype(str).replace('.0', '')
vote_2016.id_candidato_bd = vote_2016.sequencial_candidato
vote_2016 = vote_2016[vote1.columns]

In [8]:
vote = pd.concat([vote1[vote1.ano != 2016], vote_2016])

vote = vote.reset_index(drop = True)

In [9]:
vote.head()

,ano,turno,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,cargo,sigla_partido,numero_candidato,sequencial_candidato,id_candidato_bd,resultado,votos
0,1996,1,eleicao ordinaria,AL,2704302,27855,prefeito,PSD,41,NaN,NaN,nao eleito,2388
1,1996,1,eleicao ordinaria,AM,1302603,2550,prefeito,PSTU,16,NaN,NaN,nao eleito,1713
2,1996,1,eleicao ordinaria,BA,2927408,38490,prefeito,PAN,26,NaN,NaN,nao eleito,15721
3,1996,1,eleicao ordinaria,BA,2927408,38490,prefeito,PCO,29,NaN,NaN,nao eleito,705
4,1996,1,eleicao ordinaria,BA,2927408,38490,prefeito,PGT,30,NaN,NaN,nao eleito,989


In [10]:
vote.tail()

,ano,turno,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,cargo,sigla_partido,numero_candidato,sequencial_candidato,id_candidato_bd,resultado,votos
95163,2016,2,eleicao ordinaria,SP,3548708,424500,prefeito,PSDB,45,250000010056,250000010056,eleito,213661
95164,2016,2,eleicao ordinaria,SP,3552205,428730,prefeito,DEM,25,250000005826,250000005826,eleito,182833
95165,2016,2,eleicao ordinaria,SP,3552205,428730,prefeito,PSOL,50,250000004952,250000004952,nao eleito,129784
95166,2016,2,eleicao ordinaria,SP,3552502,143020,prefeito,PTB,14,250000023842,250000023842,nao eleito,56186
95167,2016,2,eleicao ordinaria,SP,3552502,143020,prefeito,PR,22,250000035239,250000035239,eleito,82912


In [11]:
#for 1996 we need the winners and then we need to find these winners in 2000. 
#we need to know if they won in 1996 and then in 2000 because then the seat is limited in 2004

#find all winners in 1996 (we have year, munic_id, munic name, candidate name)
#check if they won in 2000 (have year, munic id, *munic name, *candidate name)
#this means a seat is term limited in 2004.

#! Will the numero_candidato be the same in 2000 as it was in 1996? I'm assuming it won't be.

In [12]:
#have winners from 2000 data by name with muni name. need to get to code for municipality. 
#have winners from 1996 data by name with muni name. need to also get these codes for munics
#can then check if those winners are the same as the ones that won in 1996

In [13]:
convert_muni = pd.read_csv('https://raw.githubusercontent.com/ekatovich/Municipality_Name_ID_crosswalk/master/Raw_Brazil_GeographicalUnits.csv', encoding='latin-1')
convert_muni = convert_muni[['uf', 'uf_name', 'mesoregion_stub', 'mesoregion_name',
       'microregion_stub', 'microregion_name', 'munic_stub', 'munic_code',
       'munic_name']]
convert_muni.munic_code = convert_muni.munic_code.astype(str)
convert_muni.head()

,uf,uf_name,mesoregion_stub,mesoregion_name,microregion_stub,microregion_name,munic_stub,munic_code,munic_name
0,11,Rondônia,2,Leste Rondoniense,6,Cacoal,15,1100015,Alta Floresta D'Oeste
1,11,Rondônia,2,Leste Rondoniense,6,Cacoal,379,1100379,Alto Alegre dos Parecis
2,11,Rondônia,2,Leste Rondoniense,3,Ariquemes,403,1100403,Alto Paraíso
3,11,Rondônia,2,Leste Rondoniense,5,Alvorada D'Oeste,346,1100346,Alvorada D'Oeste
4,11,Rondônia,2,Leste Rondoniense,3,Ariquemes,23,1100023,Ariquemes


In [14]:
#get all that won in 1996. 

df_only_1996 = df[(df.ano == 1996)]
vote_only_1996 = vote[(vote.resultado == 'eleito') & (vote.ano == 1996)]
winners_1996 = pd.merge(vote_only_1996, df_only_1996, how = 'left', left_on = ['ano', 'id_municipio', 'numero_candidato'], right_on = ['ano', 'id_municipio', 'numero'])

#add muni name to winners data
winners_1996 = pd.merge(winners_1996, convert_muni[['munic_code', 'munic_name']], how = 'left', left_on = 'id_municipio', right_on = 'munic_code')

#get the muni names of 1996 winners
names_1996_turno1 = pd.read_csv('Resultado_da_Eleição_(Por_Municipio)_1996.csv', header = 1)
names_1996_turno2 = pd.read_csv('Resultado_da_Eleição_(Por_Municipio)_1996Turno2.csv', header = 1)

names_1996 = pd.concat([names_1996_turno1, names_1996_turno2])
names_1996 = names_1996[names_1996['Situação'] == 'Eleito']

names_1996 = names_1996[['Município', 'Candidato','Votos Nominais', 'Situação']]

names_1996['Município'] = names_1996['Município'].apply(unidecode).apply(lambda x: x.lower())
winners_1996.munic_name = winners_1996.munic_name.apply(unidecode).apply(lambda x: x.lower())

names_1996['Candidato'] = names_1996['Candidato'].apply(unidecode).apply(lambda x: x.lower())
#winners_1996.Candidato = winners_1996.Candidato.apply(unidecode).apply(lambda x: x.lower())


#get the names of the winners of all elections in 1996
winners_1996_names = pd.merge(winners_1996, names_1996, how = 'left', left_on = 'munic_name', right_on = 'Município')
winners_1996_names['won_1996'] = np.ones(winners_1996_names.shape[0])


names_2002_turno1 = pd.read_csv('Resultado_da_Eleição_(Por_Municipio)_2000.csv')
names_2002_turno2 = pd.read_csv('Resultado_da_Eleição_(Por_Municipio)_2000-2.csv')

names_2002 = pd.concat([names_2002_turno1, names_2002_turno2])

names_2002 = names_2002[names_2002['Situação'] == 'Eleito']
names_2002.Município = names_2002.Município.apply(unidecode).apply(lambda x: x.lower())
names_2002.Candidato = names_2002.Candidato.apply(unidecode).apply(lambda x: x.lower())

winners_1996_subset = winners_1996_names[['id_municipio', 'munic_name', 'Candidato', 'won_1996']]

term_limited_2004 = pd.merge( winners_1996_subset, names_2002, how = 'inner', left_on = ['Candidato'], right_on = ['Candidato']).id_municipio


In [15]:
#keep people that are marked as elected 
#if cand elected, what are the categories of situation variable to figure out why we're dropping winners
#situacao variable


#typically drop supp elections, happen after some unique event, doesn't show competitiveness
#here we care about demographics etc of people in office

In [16]:
vote[(vote.id_municipio == '1700251')&(vote.ano == 2016)]

,ano,turno,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,cargo,sigla_partido,numero_candidato,sequencial_candidato,id_candidato_bd,resultado,votos
94678,2016,1,eleicao ordinaria,TO,1700251,73008,prefeito,PMDB,15,270000008182,270000008182,nao eleito,605
94679,2016,1,eleicao ordinaria,TO,1700251,73008,prefeito,PSB,40,270000007820,270000007820,nao eleito,303
94680,2016,1,eleicao ordinaria,TO,1700251,73008,prefeito,PV,43,270000002720,270000002720,eleito,857


In [17]:
#sub judice check, make sure all have results
sj = df[df.situacao == 'sub judice'][['id_municipio', 'ano', 'id_candidato_bd']]
sj_merged = vote[['id_municipio', 'ano', 'id_candidato_bd', 'resultado']].merge(sj, left_on=['id_municipio', 'ano', 'id_candidato_bd'], right_on=['id_municipio', 'ano', 'id_candidato_bd'])

#drop no result values of sub judice
df = df.drop(sj_merged[sj_merged.resultado == 'renuncia/falecimento/cassacao'].index)

#repeat for pendente
pj = df[df.situacao.isin(['pendente de julgamento', 'substituto majoritario pendente de julgamento', 'substituto pendente de julgamento'])][['id_municipio', 'ano', 'id_candidato_bd']]
pj_merged = vote[['id_municipio', 'ano', 'id_candidato_bd', 'resultado']].merge(pj, left_on=['id_municipio', 'ano', 'id_candidato_bd'], right_on=['id_municipio', 'ano', 'id_candidato_bd'])
df = df.drop(pj_merged[pj_merged.resultado == 'renuncia/falecimento/cassacao'].index)




In [18]:
df.shape

(5300, 27)

In [19]:
df = df[df.tipo_eleicao != 'eleicao suplementar camamu-ba']

In [20]:
#vote[['ano', 'id_municipio']].drop_duplicates().reset_index(drop = True)

In [21]:
vote.ano.value_counts().index[0]

2020

In [22]:
#instantiate variables that we're calculating below.

final = vote[['ano', 'id_municipio']].drop_duplicates().reset_index(drop = True)

final['runoff'] = np.zeros(final.shape[0])

final['winner_turno1'] = np.full([final.shape[0]], np.nan)
final['winner_turno2'] = np.full([final.shape[0]], np.nan)

final['totalvotes_turno1'] = np.zeros(final.shape[0])
final['totalvotes_turno2'] = np.zeros(final.shape[0])

final['margin_turno1'] = np.zeros(final.shape[0])
final['margin_turno2'] = np.zeros(final.shape[0])

final['incumbant'] = np.zeros(final.shape[0])
final['term_limited_seat'] = np.zeros(final.shape[0])





In [23]:
#deal with the 1996 nas by filling in the incumbants from earlier

for muni in term_limited_2004:
    final.loc[final[(final.ano == 2004) & (final.id_municipio == muni)].index, 'term_limited_seat'] = 1

In [24]:
#check that this is 0
sum(vote[(vote.ano == 2016) & (vote.id_municipio == 4309209)].resultado == 'eleito')

0

In [25]:
np.array(df.columns)

array(['ano', 'tipo_eleicao', 'sigla_uf', 'id_municipio',
       'id_municipio_tse', 'id_candidato_bd', 'cpf', 'titulo_eleitoral',
       'sequencial', 'numero', 'nome', 'nome_urna', 'numero_partido',
       'sigla_partido', 'cargo', 'situacao', 'ocupacao',
       'data_nascimento', 'idade', 'genero', 'instrucao', 'estado_civil',
       'nacionalidade', 'sigla_uf_nascimento', 'municipio_nascimento',
       'email', 'raca'], dtype=object)

In [26]:
sum(vote_2016['sequencial_candidato'].isna())

0

In [27]:
final.ano.value_counts()

2012    5568
2016    5564
2004    5561
2008    5556
2000    5555
2020    5554
1996     116
Name: ano, dtype: int64

In [28]:
#final = final[final.ano == 2016].reset_index()

In [29]:
missing_munis = []
missing_muni_years = []

In [30]:
for i in tqdm(range(final.shape[0])):
#for i in tqdm(range(10)):
#for i in range(1000):
    #print(i)
    this_year = final.iloc[i].ano

    #if this_year == 1996:
    #    continue
    id_variable_this_year = 'id_candidato_bd'
    id_variable_last = 'id_candidato_bd'


    
    #if this_year == 2016:
    #    id_variable_this_year = 'sequencial_candidato'
    
    #if this_year == 2020:
    #    id_variable_last= 'sequencial_candidato'
    #    continue


    this_mun = final.iloc[i].id_municipio

    this_vote = vote[(vote.ano == this_year) & (vote.id_municipio == this_mun)]
    
    
    last_election = vote[(vote.ano == this_year - 4) & (vote.id_municipio == this_mun)]
    last_election = last_election[~last_election[id_variable_last].isna()]
    
    last_last_election = vote[(vote.ano == this_year - 8) & (vote.id_municipio == this_mun)]
    last_election_winner = last_election[(last_election.resultado == 'eleito') | (last_election.resultado == 'eleito por qp')]
    last_election_winner = last_election_winner[~last_election_winner[id_variable_last].isna()]
    
    winner4yr = last_election[last_election.resultado == 'eleito']
    winner4yr = winner4yr[winner4yr.tipo_eleicao == 'eleicao ordinaria'][id_variable_last]
    winner4yr = winner4yr[~winner4yr.isna()]
    
    winner8yr = last_last_election[last_last_election.resultado == 'eleito']
    winner8yr = winner8yr[winner8yr.tipo_eleicao == 'eleicao ordinaria'].id_candidato_bd
    
    if ('eleito' not in np.unique(this_vote.resultado))&('eleito por qp' not in np.unique(this_vote.resultado)):
        if this_year != 1996:

            #print('No winner for ' + str(this_mun)  + ' in ' + str(this_year))
            missing_munis.append(this_mun)
            missing_muni_years.append(this_year)

            continue
    
    
    #eventually take these out!
    if (this_year == 1996.0) & (this_mun ==  '3106200'):
        continue
    if (this_year == 1996.0) & (this_mun ==  '2408102'):
        continue
    if (this_year == 2012.0) & (this_mun == '2905800'):
        continue
    #if (this_year == 2012 )& (this_mun == '2905800'):
        #continue #eleicao suplementar camamu-ba??
    if (this_year == 2020) & (this_mun == '2205516'):
        continue #there is no winner for this year
    if (this_year == 2020) & (this_mun == '3140001'):
        continue
    
    #for 3301009 in 2004 this is needed
    if 1 not in np.unique(this_vote.turno):
        print("No Turno 1 data (only turno 2) for " + str(this_mun) + ' in ' + str(this_year))
        final.loc[i, 'runoff'] = 1
        #final.loc[i,'winner_turno1'] = this_vote.loc[this_vote[(this_vote.turno == 1) & (this_vote.resultado == '2º turno')].votos.idxmax()].id_candidato_bd
        final.loc[i,'winner_turno2'] = this_vote[(this_vote.turno == 2) & ((this_vote.resultado == 'eleito')|(this_vote.resultado == 'eleito por qp'))][id_variable_this_year].values[0]

        #final.loc[i,'totalvotes_turno1'] = sum(this_vote[this_vote.turno == 1].votos)
        final.loc[i,'totalvotes_turno2'] = sum(this_vote[this_vote.turno == 2].votos)

        #vote_sort1 = sorted(this_vote[this_vote.turno == 1].votos, reverse = True) + [0]
        #final.loc[i, 'margin_turno1'] = vote_sort1[0] - vote_sort1[1]
        vote_sort2 = sorted(this_vote[this_vote.turno == 2].votos, reverse = True) + [0]
        final.loc[i, 'margin_turno2'] = vote_sort2[0] - vote_sort2[1]

        if last_election_winner.shape[0] == 1:
            final.loc[i, 'incumbant'] = int((last_election[(last_election.resultado == 'eleito') | (last_election.resultado == 'eleito por qp')][id_variable_last]).isin(this_vote[id_variable_this_year]))

        if (last_election_winner.shape[0] != 0) & (last_election_winner.shape[0] != 1):
            print('Index ' + str(i) + ' has multiple winners. Types:' + str((last_election_winner[(~last_election_winner.tipo_eleicao.isna()) & (last_election_winner.tipo_eleicao != 'eleicao ordinaria')].tipo_eleicao).values))

        if (winner4yr.shape[0] != 0) & (winner8yr.shape[0] != 0):
            if (float(winner4yr) == float(winner8yr)):
                final.loc[i, 'term_limited_seat'] = 1
        continue
            
    #check if there is a runoff
    if 2 in np.unique(this_vote.turno):
        final.loc[i, 'runoff'] = 1
        final.loc[i,'winner_turno1'] = this_vote.loc[this_vote[(this_vote.turno == 1) & ((this_vote.resultado == '2º turno')|(this_vote.resultado == '2o turno'))].votos.idxmax()][id_variable_this_year]
        final.loc[i,'winner_turno2'] = this_vote[(this_vote.turno == 2) & ((this_vote.resultado == 'eleito')|(this_vote.resultado == 'eleito por qp'))][id_variable_this_year].values[0]
        
        final.loc[i,'totalvotes_turno1'] = sum(this_vote[this_vote.turno == 1].votos)
        final.loc[i,'totalvotes_turno2'] = sum(this_vote[this_vote.turno == 2].votos)
        
        vote_sort1 = sorted(this_vote[this_vote.turno == 1].votos, reverse = True) + [0]
        final.loc[i, 'margin_turno1'] = vote_sort1[0] - vote_sort1[1]
        vote_sort2 = sorted(this_vote[this_vote.turno == 2].votos, reverse = True) + [0]
        final.loc[i, 'margin_turno2'] = vote_sort2[0] - vote_sort2[1]
        
        if last_election_winner.shape[0] == 1:
            final.loc[i, 'incumbant'] = int((last_election[(last_election.resultado == 'eleito') | (last_election.resultado == 'eleito por qp')][id_variable_last]).isin(this_vote[id_variable_this_year]))
        
        if (last_election_winner.shape[0] != 0) & (last_election_winner.shape[0] != 1):
            print('Index ' + str(i) + ' has multiple winners. Types:' + str((last_election_winner[(~last_election_winner.tipo_eleicao.isna()) & (last_election_winner.tipo_eleicao != 'eleicao ordinaria')].tipo_eleicao).values))
        
        if (winner4yr.shape[0] != 0) & (winner8yr.shape[0] != 0):
            if (float(winner4yr) == float(winner8yr)):
                final.loc[i, 'term_limited_seat'] = 1
            
    #if no runoff
    if 2 not in np.unique(this_vote.turno):
        #final.loc[i, 'runoff'] = 0
        #final.loc[i,'winner_turno1'] = this_vote.loc[this_vote[(this_vote.turno == 1) & (this_vote.resultado == 'eleito')].votos.idxmax()][id_variable_this_year]
        final.loc[i,'winner_turno1'] = this_vote.loc[this_vote[(this_vote.turno == 1) & ((this_vote.resultado == 'eleito')|(this_vote.resultado == 'eleito por qp'))].votos.idxmax()][id_variable_this_year]
        #final.loc[i,'winner_turno2'] = np.nan
        
        final.loc[i,'totalvotes_turno1'] = sum(this_vote[this_vote.turno == 1].votos)
        #final.loc[i,'totalvotes_turno2'] = sum(this_vote[this_vote.turno == 2].votos)
        
        vote_sort1 = sorted(this_vote[this_vote.turno == 1].votos, reverse = True) + [0]
        final.loc[i, 'margin_turno1'] = vote_sort1[0] - vote_sort1[1]
        
        if last_election_winner.shape[0] == 1:
            final.loc[i, 'incumbant'] = int((last_election[(last_election.resultado == 'eleito') | (last_election.resultado == 'eleito por qp')][id_variable_last]).isin(this_vote))
        
        #sometimes are two win options?
        if (last_election_winner.shape[0] != 0) & (last_election_winner.shape[0] != 1):
            print('Index ' + str(i) + ' has multiple winners. Types:' + str((last_election_winner[(~last_election_winner.tipo_eleicao.isna()) & (last_election_winner.tipo_eleicao != 'eleicao ordinaria')].tipo_eleicao).values))

            
        if (winner4yr.shape[0] != 0) & (winner8yr.shape[0] != 0):
            if (float(winner4yr) == float(winner8yr)):
                final.loc[i, 'term_limited_seat'] = 1   

 31%|███       | 10288/33474 [04:21<10:07, 38.15it/s]

No Turno 1 data (only turno 2) for 3301009 in 2004


 72%|███████▏  | 23943/33474 [10:04<03:45, 42.18it/s]

No Turno 1 data (only turno 2) for 1302603 in 2020


100%|██████████| 33474/33474 [14:15<00:00, 39.13it/s]


In [31]:
missing_data = pd.DataFrame(
    {'year': missing_muni_years,
     'muni': missing_munis
    })


In [32]:
missing_data.head()

,year,muni
0,2020,3140001
1,2020,2205516
2,2020,3548807
3,2020,2910909
4,2020,3303906


In [33]:
missing_data = missing_data.drop_duplicates(keep = 'first')

In [34]:
missing_data.to_csv('MissingElectionData.csv', index = False)

In [35]:
filter_muni = '2803609'
filter_year = 2016

vote1[(vote1.ano == filter_year)&(vote1.id_municipio == filter_muni)]


,ano,turno,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,cargo,sigla_partido,numero_candidato,sequencial_candidato,id_candidato_bd,resultado,votos
41268,2016,1,eleicao ordinaria,SE,2803609,31712,prefeito,MDB,15,260000001505,502673,nao eleito,8337
64093,2016,1,eleicao ordinaria,SE,2803609,31712,prefeito,DEM,25,260000001310,1325512,nao eleito,0


In [78]:
final.to_csv('ElectionResults.csv', index = False)

In [36]:
final.head()

,ano,id_municipio,runoff,winner_turno1,winner_turno2,totalvotes_turno1,totalvotes_turno2,margin_turno1,margin_turno2,incumbant,term_limited_seat
0,1996,2704302,1.0,NaN,NaN,218517.0,219899.0,6723.0,4347.0,0.0,0.0
1,1996,1302603,1.0,NaN,NaN,468395.0,468458.0,25688.0,2558.0,0.0,0.0
2,1996,2927408,0.0,NaN,NaN,791126.0,0.0,171384.0,0.0,0.0,0.0
3,1996,3205309,0.0,NaN,NaN,145863.0,0.0,47981.0,0.0,0.0,0.0
4,1996,5200050,0.0,NaN,NaN,1657.0,0.0,211.0,0.0,0.0,0.0


In [39]:
missing_data.tail()

,year,muni
105,2016,3539103
106,2016,3545506
107,2016,3546306
108,2016,3551306
109,2016,3555307


In [40]:
filter_muni = '3539103'
filter_year = 2016

vote1[(vote1.ano == filter_year)&(vote1.id_municipio == filter_muni)]


,ano,turno,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,cargo,sigla_partido,numero_candidato,sequencial_candidato,id_candidato_bd,resultado,votos
14759,2016,1,eleicao ordinaria,SP,3539103,68837,prefeito,PDT,12,250000062374,1681584,nao eleito,235
29496,2016,1,eleicao ordinaria,SP,3539103,68837,prefeito,PTB,14,250000056581,1251198,nao eleito,0
41424,2016,1,eleicao ordinaria,SP,3539103,68837,prefeito,MDB,15,250000023247,1715925,nao eleito,4227
47386,2016,1,eleicao ordinaria,SP,3539103,68837,prefeito,PODE,19,250000052651,1239109,nao eleito,291
67045,2016,1,eleicao ordinaria,SP,3539103,68837,prefeito,PRTB,28,250000063596,1459709,nao eleito,21
88180,2016,1,eleicao ordinaria,SP,3539103,68837,prefeito,PSOL,50,250000044104,1359826,nao eleito,125


In [41]:
pathvote = '/Users/annieulichney/Library/CloudStorage/Box-Box/Deforestation_MachineLearning/Data/cand_prefeito_vote.Rdata'

orig = pyreadr.read_r(pathvote)['cand']

In [42]:
orig.head()

,ano,turno,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,cargo,sigla_partido,numero_candidato,sequencial_candidato,id_candidato_bd,resultado,votos
0,1996,1,eleicao ordinaria,AL,2704302,27855,prefeito,PSD,41,NaN,NaN,nao eleito,2388
1,1996,1,eleicao ordinaria,AM,1302603,2550,prefeito,PSTU,16,NaN,NaN,nao eleito,1713
2,1996,1,eleicao ordinaria,BA,2927408,38490,prefeito,PAN,26,NaN,NaN,nao eleito,15721
3,1996,1,eleicao ordinaria,BA,2927408,38490,prefeito,PCO,29,NaN,NaN,nao eleito,705
4,1996,1,eleicao ordinaria,BA,2927408,38490,prefeito,PGT,30,NaN,NaN,nao eleito,989


In [32]:
this_mun

'2708808'

In [218]:
winner4yr

88676      20000001553
489782    250000050616
Name: id_candidato_bd, dtype: object

In [219]:
winner8yr

6554    591731
Name: id_candidato_bd, dtype: object

In [221]:
#vote.to_csv('vote_updated.csv')

In [97]:
final.head()

,index,ano,id_municipio,runoff,winner_turno1,winner_turno2,totalvotes_turno1,totalvotes_turno2,margin_turno1,margin_turno2,incumbant,term_limited_seat
0,27910,2016.0,1200013,0.0,1.000000e+10,NaN,13947.0,0.0,2307.0,0.0,0.0,0.0
1,27911,2016.0,1200054,0.0,1.000000e+10,NaN,8560.0,0.0,982.0,0.0,0.0,0.0
2,27912,2016.0,1200104,0.0,1.000000e+10,NaN,25022.0,0.0,806.0,0.0,0.0,0.0
3,27913,2016.0,1200138,0.0,1.000000e+10,NaN,12036.0,0.0,385.0,0.0,0.0,0.0
4,27914,2016.0,1200179,0.0,1.000000e+10,NaN,11932.0,0.0,300.0,0.0,0.0,0.0


In [98]:
final.shape

(5566, 12)

In [73]:
final[~final.winner_turno1.isna()]



,ano,id_municipio,runoff,winner_turno1,winner_turno2,totalvotes_turno1,totalvotes_turno2,margin_turno1,margin_turno2,incumbant,term_limited_seat
0,NaN,NaN,NaN,1.000000e+10,NaN,13947.0,NaN,2307.0,NaN,0.0,NaN
1,NaN,NaN,NaN,1.000000e+10,NaN,8560.0,NaN,982.0,NaN,0.0,NaN
2,NaN,NaN,NaN,1.000000e+10,NaN,25022.0,NaN,806.0,NaN,0.0,NaN
3,NaN,NaN,NaN,1.000000e+10,NaN,12036.0,NaN,385.0,NaN,0.0,NaN
4,NaN,NaN,NaN,1.000000e+10,NaN,11932.0,NaN,300.0,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
5561,NaN,NaN,NaN,2.700000e+11,NaN,22789.0,NaN,890.0,NaN,0.0,1.0
5562,NaN,NaN,NaN,2.700000e+11,NaN,3169.0,NaN,99.0,NaN,0.0,NaN
5563,NaN,NaN,NaN,2.700000e+11,NaN,3181.0,NaN,450.0,NaN,0.0,NaN
5564,NaN,NaN,NaN,2.700000e+11,NaN,10950.0,NaN,2904.0,NaN,0.0,NaN


In [389]:
final[final.ano == 2016]

,ano,id_municipio,runoff,winner_turno1,winner_turno2,totalvotes_turno1,totalvotes_turno2,margin_turno1,margin_turno2,incumbant,term_limited_seat
27910,2016.0,1200013.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
27911,2016.0,1200054.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
27912,2016.0,1200104.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
27913,2016.0,1200138.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
27914,2016.0,1200179.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
33465,2016.0,1721109.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
33466,2016.0,1721257.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
33467,2016.0,1721307.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
33468,2016.0,1722081.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0


In [382]:
this_vote[(this_vote.turno == 1) & (this_vote.resultado == 'eleito')]

,ano,turno,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,cargo,sigla_partido,numero_candidato,sequencial_candidato,id_candidato_bd,resultado,votos


In [385]:
np.unique(this_vote.resultado)

array(['eleito por media', 'eleito por qp', 'nao eleito', 'suplente'],
      dtype=object)

In [381]:
this_vote.loc[this_vote[(this_vote.turno == 1) & ((this_vote.resultado == 'eleito')|(this_vote.resultado == 'eleito por qp'))].votos.idxmax()][id_variable_this_year]

ValueError: attempt to get argmax of an empty sequence

In [299]:
this_vote.loc[this_vote[(this_vote.turno == 1) & ((this_vote.resultado == '2º turno')|(this_vote.resultado == '2o turno'))].votos.idxmax()][id_variable_this_year]

90000014463

In [298]:
np.unique(this_vote[(this_vote.turno == 1)].resultado)

array(['2o turno', 'eleito', 'eleito por media', 'eleito por qp',
       'nao eleito', 'suplente'], dtype=object)

In [293]:
this_vote.turno

79495      1
79496      1
79497      1
79498      1
79499      1
          ..
1104401    2
1104402    2
1104403    2
1104404    2
1104405    2
Name: turno, Length: 1024911, dtype: int64

In [422]:

for i in tqdm(range(final.shape[0])):
#for i in range(1000):
    #print(i)
    this_year = final.iloc[i].ano

    #if this_year == 1996:
    #    continue
    id_variable = 'id_candidato_bd'
    if this_year == 2016:
        id_variable = 'sequencial_candidato'
    
    if this_year == 2020:
        continue

    this_mun = final.iloc[i].id_municipio

    this_vote = vote[(vote.ano == this_year) & (vote.id_municipio == this_mun)]
    
    last_election = vote[(vote.ano == this_year - 4) & (vote.id_municipio == this_mun)]
    last_election = last_election[~last_election.id_candidato_bd.isna()]
    last_last_election = vote[(vote.ano == this_year - 8) & (vote.id_municipio == this_mun)]
    last_election_winner = last_election[(last_election.resultado == 'eleito') | (last_election.resultado == 'eleito por qp')]
    last_election_winner = last_election_winner[~last_election_winner.id_candidato_bd.isna()]
    
    winner4yr = last_election[last_election.resultado == 'eleito']
    winner4yr = winner4yr[winner4yr.tipo_eleicao == 'eleicao ordinaria'].id_candidato_bd
    winner4yr = winner4yr[~winner4yr.isna()]
    
    winner8yr = last_last_election[last_last_election.resultado == 'eleito']
    winner8yr = winner8yr[winner8yr.tipo_eleicao == 'eleicao ordinaria'].id_candidato_bd
    
    if ('eleito' not in np.unique(this_vote.resultado))&('eleito por qp' not in np.unique(this_vote.resultado)):
        if this_year != 1996:

            print('No winner for ' + str(this_mun)  + ' in ' + str(this_year))
            continue
    
    
    #eventually take these out!
    if (this_year == 1996.0) & (this_mun ==  '3106200'):
        continue
    if (this_year == 1996.0) & (this_mun ==  '2408102'):
        continue
    if (this_year == 2012.0) & (this_mun == '2905800'):
        continue
    #if (this_year == 2012 )& (this_mun == '2905800'):
        #continue #eleicao suplementar camamu-ba??
    if (this_year == 2020) & (this_mun == '2205516'):
        continue #there is no winner for this year
    if (this_year == 2020) & (this_mun == '3140001'):
        continue
    
    #for 3301009 in 2004 this is needed
    if 1 not in np.unique(this_vote.turno):
        print("No Turno 1 data (only turno 2) for " + str(this_mun) + ' in ' + str(this_year))
        final.loc[i, 'runoff'] = 1
        #final.loc[i,'winner_turno1'] = this_vote.loc[this_vote[(this_vote.turno == 1) & (this_vote.resultado == '2º turno')].votos.idxmax()].id_candidato_bd
        final.loc[i,'winner_turno2'] = this_vote[(this_vote.turno == 2) & ((this_vote.resultado == 'eleito')|(this_vote.resultado == 'eleito por qp'))].id_candidato_bd.values[0]

        #final.loc[i,'totalvotes_turno1'] = sum(this_vote[this_vote.turno == 1].votos)
        final.loc[i,'totalvotes_turno2'] = sum(this_vote[this_vote.turno == 2].votos)

        #vote_sort1 = sorted(this_vote[this_vote.turno == 1].votos, reverse = True) + [0]
        #final.loc[i, 'margin_turno1'] = vote_sort1[0] - vote_sort1[1]
        vote_sort2 = sorted(this_vote[this_vote.turno == 2].votos, reverse = True) + [0]
        final.loc[i, 'margin_turno2'] = vote_sort2[0] - vote_sort2[1]

        if last_election_winner.shape[0] == 1:
            final.loc[i, 'incumbant'] = int((last_election[(last_election.resultado == 'eleito') | (last_election.resultado == 'eleito por qp')].id_candidato_bd).isin(this_vote.id_candidato_bd))

        if (last_election_winner.shape[0] != 0) & (last_election_winner.shape[0] != 1):
            print('Index ' + str(i) + ' has multiple winners. Types:' + str((last_election_winner[(~last_election_winner.tipo_eleicao.isna()) & (last_election_winner.tipo_eleicao != 'eleicao ordinaria')].tipo_eleicao).values))

        if (winner4yr.shape[0] != 0) & (winner8yr.shape[0] != 0):
            if (float(winner4yr) == float(winner8yr)):
                final.loc[i, 'term_limited_seat'] = 1
        continue
            
    #check if there is a runoff
    if 2 in np.unique(this_vote.turno):
        final.loc[i, 'runoff'] = 1
        final.loc[i,'winner_turno1'] = this_vote.loc[this_vote[(this_vote.turno == 1) & (this_vote.resultado == '2º turno')].votos.idxmax()].id_candidato_bd
        final.loc[i,'winner_turno2'] = this_vote[(this_vote.turno == 2) & ((this_vote.resultado == 'eleito')|(this_vote.resultado == 'eleito por qp'))].id_candidato_bd.values[0]
        
        final.loc[i,'totalvotes_turno1'] = sum(this_vote[this_vote.turno == 1].votos)
        final.loc[i,'totalvotes_turno2'] = sum(this_vote[this_vote.turno == 2].votos)
        
        vote_sort1 = sorted(this_vote[this_vote.turno == 1].votos, reverse = True) + [0]
        final.loc[i, 'margin_turno1'] = vote_sort1[0] - vote_sort1[1]
        vote_sort2 = sorted(this_vote[this_vote.turno == 2].votos, reverse = True) + [0]
        final.loc[i, 'margin_turno2'] = vote_sort2[0] - vote_sort2[1]
        
        if last_election_winner.shape[0] == 1:
            final.loc[i, 'incumbant'] = int((last_election[(last_election.resultado == 'eleito') | (last_election.resultado == 'eleito por qp')].id_candidato_bd).isin(this_vote.id_candidato_bd))
        
        if (last_election_winner.shape[0] != 0) & (last_election_winner.shape[0] != 1):
            print('Index ' + str(i) + ' has multiple winners. Types:' + str((last_election_winner[(~last_election_winner.tipo_eleicao.isna()) & (last_election_winner.tipo_eleicao != 'eleicao ordinaria')].tipo_eleicao).values))
        
        if (winner4yr.shape[0] != 0) & (winner8yr.shape[0] != 0):
            if (float(winner4yr) == float(winner8yr)):
                final.loc[i, 'term_limited_seat'] = 1
            
    #if no runoff
    if 2 not in np.unique(this_vote.turno):
        #final.loc[i, 'runoff'] = 0
        final.loc[i,'winner_turno1'] = this_vote.loc[this_vote[(this_vote.turno == 1) & (this_vote.resultado == 'eleito')].votos.idxmax()].id_candidato_bd
        #final.loc[i,'winner_turno2'] = np.nan
        
        final.loc[i,'totalvotes_turno1'] = sum(this_vote[this_vote.turno == 1].votos)
        #final.loc[i,'totalvotes_turno2'] = sum(this_vote[this_vote.turno == 2].votos)
        
        vote_sort1 = sorted(this_vote[this_vote.turno == 1].votos, reverse = True) + [0]
        final.loc[i, 'margin_turno1'] = vote_sort1[0] - vote_sort1[1]
        
        if last_election_winner.shape[0] == 1:
            final.loc[i, 'incumbant'] = int((last_election[(last_election.resultado == 'eleito') | (last_election.resultado == 'eleito por qp')].id_candidato_bd).isin(this_vote.id_candidato_bd))
        
        #sometimes are two win options?
        if (last_election_winner.shape[0] != 0) & (last_election_winner.shape[0] != 1):
            print('Index ' + str(i) + ' has multiple winners. Types:' + str((last_election_winner[(~last_election_winner.tipo_eleicao.isna()) & (last_election_winner.tipo_eleicao != 'eleicao ordinaria')].tipo_eleicao).values))

            
        if (winner4yr.shape[0] != 0) & (winner8yr.shape[0] != 0):
            if (float(winner4yr) == float(winner8yr)):
                final.loc[i, 'term_limited_seat'] = 1   

  1%|          | 35/5560 [00:07<18:43,  4.92it/s]


ValueError: attempt to get argmax of an empty sequence

In [421]:
final = final[final.ano == 2016]

In [423]:
final.head()

,ano,id_municipio,runoff,winner_turno1,winner_turno2,totalvotes_turno1,totalvotes_turno2,margin_turno1,margin_turno2,incumbant,term_limited_seat
27910,2016.0,1200013.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
27911,2016.0,1200054.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
27912,2016.0,1200104.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
27913,2016.0,1200138.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
27914,2016.0,1200179.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0


In [130]:
#function to assign employment category to final cleaned dataframe
def occupation_label(occ):  

           
    agriculture = ['tecnico em agronomia e agrimensura', 'trabalhador agricola', 'trabalhador da pecuaria', 
                  'agricultor', 'agronomo', 'aposentado (exceto funcionario publico)',
                   'aposentado (exceto servidor publico)', 'operador de implemento de agricultura pecuaria e exploracao florestal', 
                  'pecuarista', 'produtor agropecuario', 'proprietario de estabelecimento agricola da pecuaria e florestal']

    construction_industrial = ['tecnico de obras civis estradas saneamento e trabalhador assemelhado', 
                    'tecnico em edificacoes', 'torneiro mecanico', 'trabalhador de construcao civil', 
                   'trabalhador metalurgico e siderurgico', 'eletricista e assemelhados', 
                    'encanadorsoldadorchapeadorcaldeireiromontador de estrutura metalic', 'engenheiro', 
                   'industrial', 'mecanico de manutencao', 'operador de aparelhos de producao industrial'
                   'proprietario de estabelecimento de prestacao de servicos']

    politics = ['membros do poder executivo: presidente ministro governador prefeito', 
                'membros do poder legislativo: senador deputados e vereador', 'prefeito',
               'presidente da republica ministro de estado governador e prefeito',
               'senador deputado e vereador']

    business = ['bancario e economiario', 'capitalista de ativos financeiros', 'comerciario', 
                  'corretor de imoveis seguros titulos e valores', 'empresario', 
                  'proprietario de estabelecimento comercial']

    public_employee = ['agente postal', 'assistente social', 'bombeiro civil', 'bombeiro militar',
                      'funcionario publico civil aposentado', 'membro das forcas armadas', 
                       'membro do ministerio publico', 'militar reformado', 'ocupante de cargo em comissao', 
                      'policial civil', 'policial militar', 'serventuario de justica', 'servidor publico civil aposentado', 
                      'servidor publico estadual', 'servidor publico federal', 'servidor publico municipal']

    mining_oil = ["tecnico de mineracao metalurgia e geologia", 'petroleiro']
    
    if occ in (agriculture):
        return "agriculture"
    if occ in (construction_industrial):
        return "construction_industrial"
    if occ in (politics):
        return "politics"
    if occ in (business):
        return "business"
    if occ in (public_employee):
        return "public_employee"
    else: return np.nan


In [131]:
final.head()

,ano,id_municipio,runoff,winner_turno1,winner_turno2,totalvotes_turno1,totalvotes_turno2,margin_turno1,margin_turno2,incumbant,term_limited_seat
0,1996,2704302,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
1,1996,1302603,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
2,1996,2927408,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
3,1996,3205309,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
4,1996,5200050,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0


In [132]:
df.head()

,ano,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,id_candidato_bd,cpf,titulo_eleitoral,sequencial,numero,...,data_nascimento,idade,genero,instrucao,estado_civil,nacionalidade,sigla_uf_nascimento,municipio_nascimento,email,raca
0,1996,eleicao ordinaria,PI,2211001,12190,NaN,NaN,NaN,NaN,16,...,1966-11-18,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1996,eleicao ordinaria,SE,2800308,31054,NaN,NaN,NaN,NaN,16,...,1956-05-23,40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1996,eleicao ordinaria,RS,4314902,88013,NaN,NaN,NaN,NaN,56,...,1950-07-12,46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1996,eleicao ordinaria,MG,3106200,41238,NaN,NaN,NaN,NaN,16,...,1953-06-07,43,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1996,eleicao ordinaria,PB,2507507,20516,NaN,NaN,NaN,NaN,16,...,1953-04-04,43,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [145]:
cols_of_interest = ['id_candidato_bd', 'ocupacao', 'idade', 'genero', 'nacionalidade', 'raca', 'ano', 'id_municipio']

df_subset = df[cols_of_interest]

In [146]:
final_merge1 = pd.merge(final, df_subset, how = 'left', left_on = ['winner_turno1', 'ano', 'id_municipio'], right_on = ['id_candidato_bd', 'ano', 'id_municipio'])


final_merge1.columns = ['ano', 'id_municipio', 'runoff', 'winner_turno1', 'winner_turno2',
       'totalvotes_turno1', 'totalvotes_turno2', 'margin_turno1',
       'margin_turno2', 'incumbant', 'term_limited_seat',
       'id_candidato_bd', 'ocupacao_turno1', 'idade_turno1', 'genero_turno1', 'nacionalidade_turno1',
       'raca_turno1']

In [147]:
final_merge2 = pd.merge(final_merge1, df_subset, how = 'left', left_on = ['winner_turno2', 'ano', 'id_municipio'], right_on = ['id_candidato_bd', 'ano', 'id_municipio'])
final_merge2.columns =['ano', 'id_municipio', 'runoff', 'winner_turno1', 'winner_turno2',
       'totalvotes_turno1', 'totalvotes_turno2', 'margin_turno1',
       'margin_turno2', 'incumbant', 'term_limited_seat',
       'id_candidato_bd_x', 'ocupacao_turno1', 'idade_turno1',
       'genero_turno1', 'nacionalidade_turno1', 'raca_turno1',
       'id_candidato_bd_y', 'ocupacao_turno2', 'idade_turno2', 'genero_turno2',
       'nacionalidade_turno2', 'raca_turno2']



In [149]:
final_merge2.ocupacao_turno1 = final_merge2.ocupacao_turno1.apply(lambda x: occupation_label(x))
final_merge2.ocupacao_turno2 = final_merge2.ocupacao_turno2.apply(lambda x: occupation_label(x))

In [151]:
final.head()

,ano,id_municipio,runoff,winner_turno1,winner_turno2,totalvotes_turno1,totalvotes_turno2,margin_turno1,margin_turno2,incumbant,term_limited_seat
0,1996,2704302,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
1,1996,1302603,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
2,1996,2927408,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
3,1996,3205309,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
4,1996,5200050,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0


In [155]:
final_merge2.columns

Index(['ano', 'id_municipio', 'runoff', 'winner_turno1', 'winner_turno2',
       'totalvotes_turno1', 'totalvotes_turno2', 'margin_turno1',
       'margin_turno2', 'incumbant', 'term_limited_seat', 'id_candidato_bd_x',
       'ocupacao_turno1', 'idade_turno1', 'genero_turno1',
       'nacionalidade_turno1', 'raca_turno1', 'id_candidato_bd_y',
       'ocupacao_turno2', 'idade_turno2', 'genero_turno2',
       'nacionalidade_turno2', 'raca_turno2'],
      dtype='object')

In [154]:
final_merge2[(final_merge2.ano != 1996)&(final_merge2.ano != 2016)].head(30)

,ano,id_municipio,runoff,winner_turno1,winner_turno2,totalvotes_turno1,totalvotes_turno2,margin_turno1,margin_turno2,incumbant,...,idade_turno1,genero_turno1,nacionalidade_turno1,raca_turno1,id_candidato_bd_y,ocupacao_turno2,idade_turno2,genero_turno2,nacionalidade_turno2,raca_turno2
1174,2000,2700409,0.0,1153134,NaN,15594.0,0.0,1607.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1175,2000,2703908,0.0,11754,NaN,2290.0,0.0,300.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1176,2000,2705002,0.0,1263076,NaN,9568.0,0.0,4672.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1177,2000,2706448,0.0,NaN,NaN,5620.0,0.0,1103.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1178,2000,2707008,0.0,1700140,NaN,1460.0,0.0,279.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1179,2000,2707503,0.0,1619309,NaN,8524.0,0.0,1312.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1180,2000,2708303,0.0,562380,NaN,9283.0,0.0,1009.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1181,2000,2708907,0.0,1265591,NaN,4882.0,0.0,1412.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1182,2000,2709152,0.0,548325,NaN,14016.0,0.0,1569.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1183,2000,1300060,0.0,105602,NaN,2445.0,0.0,123.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
## 2020 muns that have no id candidato for election winner 
#np.unique(vote[(vote.ano == 2020)&(vote.id_candidato_bd.isna())].id_municipio)

In [34]:
vote.head()

,ano,turno,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,cargo,sigla_partido,numero_candidato,sequencial_candidato,id_candidato_bd,resultado,votos
0,1996,1,eleicao ordinaria,AL,2704302,27855,prefeito,PSD,41,NaN,NaN,nao eleito,2388
1,1996,1,eleicao ordinaria,AM,1302603,2550,prefeito,PSTU,16,NaN,NaN,nao eleito,1713
2,1996,1,eleicao ordinaria,BA,2927408,38490,prefeito,PAN,26,NaN,NaN,nao eleito,15721
3,1996,1,eleicao ordinaria,BA,2927408,38490,prefeito,PCO,29,NaN,NaN,nao eleito,705
4,1996,1,eleicao ordinaria,BA,2927408,38490,prefeito,PGT,30,NaN,NaN,nao eleito,989


In [35]:
#classify all supplementary elections as something new
#keep elected people in main, create second ind for supp elect that term, 
#for muns with 1, create new var saying year and who won #sup year sup id, gender party occ 

#if we don't know year can't go in and change on year level (e.g. override)

In [36]:
final.head()

,ano,id_municipio,runoff,winner_turno1,winner_turno2,totalvotes_turno1,totalvotes_turno2,margin_turno1,margin_turno2,incumbant,term_limited_seat
0,1996,2704302,1.0,NaN,NaN,218517.0,219899.0,6723.0,4347.0,0.0,0.0
1,1996,1302603,1.0,NaN,NaN,468395.0,468458.0,25688.0,2558.0,0.0,0.0
2,1996,2927408,0.0,NaN,NaN,791126.0,0.0,171384.0,0.0,0.0,0.0
3,1996,3205309,0.0,NaN,NaN,145863.0,0.0,47981.0,0.0,0.0,0.0
4,1996,5200050,0.0,NaN,NaN,1657.0,0.0,211.0,0.0,0.0,0.0


In [37]:
#this is a good demo of the 2nd turno thing

this_year = 2020
this_mun = '3140001'

vote[(vote.ano == this_year) & (vote.id_municipio == this_mun)]

,ano,turno,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,cargo,sigla_partido,numero_candidato,sequencial_candidato,id_candidato_bd,resultado,votos
1481,2020,1,eleicao ordinaria,MG,3140001,47996,prefeito,PSTU,16,130000950637,1326417,nao eleito,1847
1482,2020,1,eleicao ordinaria,MG,3140001,47996,prefeito,REDE,18,130000791232,447469,nao eleito,185
2923,2020,1,eleicao ordinaria,MG,3140001,47996,prefeito,REPUBLICANOS,10,130001244159,1019405,nao eleito,4520
30632,2020,1,eleicao ordinaria,MG,3140001,47996,prefeito,PTB,14,130001166586,547875,nao eleito,2168
57803,2020,1,eleicao ordinaria,MG,3140001,47996,prefeito,CIDADANIA,23,130000797010,1604845,nao eleito,11168


In [38]:
vote.shape[0]

95735

In [39]:
vote[(vote.ano == 1996)&(vote.id_candidato_bd.isna())]

,ano,turno,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,cargo,sigla_partido,numero_candidato,sequencial_candidato,id_candidato_bd,resultado,votos
0,1996,1,eleicao ordinaria,AL,2704302,27855,prefeito,PSD,41,NaN,NaN,nao eleito,2388
1,1996,1,eleicao ordinaria,AM,1302603,2550,prefeito,PSTU,16,NaN,NaN,nao eleito,1713
2,1996,1,eleicao ordinaria,BA,2927408,38490,prefeito,PAN,26,NaN,NaN,nao eleito,15721
3,1996,1,eleicao ordinaria,BA,2927408,38490,prefeito,PCO,29,NaN,NaN,nao eleito,705
4,1996,1,eleicao ordinaria,BA,2927408,38490,prefeito,PGT,30,NaN,NaN,nao eleito,989
...,...,...,...,...,...,...,...,...,...,...,...,...,...
94701,1996,1,eleicao ordinaria,AM,1302603,2550,prefeito,PT do B,70,NaN,NaN,nao eleito,671
94702,1996,1,eleicao ordinaria,CE,2304400,13897,prefeito,PT do B,70,NaN,NaN,nao eleito,4822
94703,1996,1,eleicao ordinaria,GO,5220454,92703,prefeito,PT do B,70,NaN,NaN,nao eleito,301
94704,1996,1,eleicao ordinaria,PE,2611606,25313,prefeito,PT do B,70,NaN,NaN,nao eleito,1406


In [40]:
#this is a good demo of the 2nd turno thing
#look online for the results in these instances

#ALEXANDRE KALIL name of candidate who won in 2020, look up 96
this_year = 1996
this_mun = '3106200'

vote[(vote.ano == this_year) & (vote.id_municipio == this_mun)]

,ano,turno,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,cargo,sigla_partido,numero_candidato,sequencial_candidato,id_candidato_bd,resultado,votos
11,1996,1,eleicao ordinaria,MG,3106200,41238,prefeito,PSTU,16,NaN,NaN,nao eleito,4692
11115,1996,1,eleicao ordinaria,MG,3106200,41238,prefeito,PDT,12,NaN,NaN,nao eleito,75849
16596,1996,1,eleicao ordinaria,MG,3106200,41238,prefeito,PT,13,NaN,NaN,nao eleito,228442
58175,1996,1,eleicao ordinaria,MG,3106200,41238,prefeito,PFL,25,NaN,NaN,nao eleito,25068
68933,1996,1,eleicao ordinaria,MG,3106200,41238,prefeito,PRN,36,NaN,NaN,nao eleito,11645
69503,1996,1,eleicao ordinaria,MG,3106200,41238,prefeito,PSB,40,NaN,NaN,2º turno,429948
76962,1996,1,eleicao ordinaria,MG,3106200,41238,prefeito,PSDB,45,NaN,NaN,2º turno,279055


In [41]:
i

999

In [42]:
i = 12046
this_year = final.iloc[i].ano
this_mun = final.iloc[i].id_municipio

this_vote = vote[(vote.ano == this_year) & (vote.id_municipio == this_mun)]

last_election = vote[(vote.ano == this_year - 4) & (vote.id_municipio == this_mun)]
last_last_election = vote[(vote.ano == this_year - 8) & (vote.id_municipio == this_mun)]
last_election_winner = last_election[(last_election.resultado == 'eleito') | (last_election.resultado == 'eleito por qp')]

winner4yr = last_election[last_election.resultado == 'eleito']
winner4yr = winner4yr[winner4yr.tipo_eleicao == 'eleicao ordinaria'].id_candidato_bd
winner4yr = winner4yr[~winner4yr.isna()]

winner8yr = last_last_election[last_last_election.resultado == 'eleito']
winner8yr = winner8yr[winner8yr.tipo_eleicao == 'eleicao ordinaria'].id_candidato_bd

In [174]:
final.loc[i, 'runoff'] = 1
#final.loc[i,'winner_turno1'] = this_vote.loc[this_vote[(this_vote.turno == 1) & (this_vote.resultado == '2º turno')].votos.idxmax()].id_candidato_bd
#final.loc[i,'winner_turno2'] = this_vote[(this_vote.turno == 2) & ((this_vote.resultado == 'eleito')|(this_vote.resultado == 'eleito por qp'))].id_candidato_bd.values[0]

#final.loc[i,'totalvotes_turno1'] = sum(this_vote[this_vote.turno == 1].votos)
#final.loc[i,'totalvotes_turno2'] = sum(this_vote[this_vote.turno == 2].votos)

# vote_sort1 = sorted(this_vote[this_vote.turno == 1].votos, reverse = True) + [0]
# final.loc[i, 'margin_turno1'] = vote_sort1[0] - vote_sort1[1]
# vote_sort2 = sorted(this_vote[this_vote.turno == 2].votos, reverse = True) + [0]
# final.loc[i, 'margin_turno2'] = vote_sort2[0] - vote_sort2[1]

# if last_election_winner.shape[0] == 1:
#     final.loc[i, 'incumbant'] = int((last_election[(last_election.resultado == 'eleito') | (last_election.resultado == 'eleito por qp')].id_candidato_bd).isin(this_vote.id_candidato_bd))

# if (last_election_winner.shape[0] != 0) & (last_election_winner.shape[0] != 1):
#     print('Index ' + str(i) + ' has multiple winners. Types:' + str((last_election_winner[(~last_election_winner.tipo_eleicao.isna()) & (last_election_winner.tipo_eleicao != 'eleicao ordinaria')].tipo_eleicao).values))

# if (winner4yr.shape[0] != 0) & (winner8yr.shape[0] != 0):
#     if (float(winner4yr) == float(winner8yr)):
#         final.loc[i, 'term_limited_seat'] = 1

In [182]:

if 1 not in np.unique(this_vote.turno):
    print("No Turno 1 data (only turno 2) for " + str(this_mun) + ' in ' + str(this_year))
    final.loc[i, 'runoff'] = 1
    #final.loc[i,'winner_turno1'] = this_vote.loc[this_vote[(this_vote.turno == 1) & (this_vote.resultado == '2º turno')].votos.idxmax()].id_candidato_bd
    final.loc[i,'winner_turno2'] = this_vote[(this_vote.turno == 2) & ((this_vote.resultado == 'eleito')|(this_vote.resultado == 'eleito por qp'))].id_candidato_bd.values[0]

    #final.loc[i,'totalvotes_turno1'] = sum(this_vote[this_vote.turno == 1].votos)
    final.loc[i,'totalvotes_turno2'] = sum(this_vote[this_vote.turno == 2].votos)

    #vote_sort1 = sorted(this_vote[this_vote.turno == 1].votos, reverse = True) + [0]
    #final.loc[i, 'margin_turno1'] = vote_sort1[0] - vote_sort1[1]
    vote_sort2 = sorted(this_vote[this_vote.turno == 2].votos, reverse = True) + [0]
    final.loc[i, 'margin_turno2'] = vote_sort2[0] - vote_sort2[1]

    if last_election_winner.shape[0] == 1:
        final.loc[i, 'incumbant'] = int((last_election[(last_election.resultado == 'eleito') | (last_election.resultado == 'eleito por qp')].id_candidato_bd).isin(this_vote.id_candidato_bd))

    if (last_election_winner.shape[0] != 0) & (last_election_winner.shape[0] != 1):
        print('Index ' + str(i) + ' has multiple winners. Types:' + str((last_election_winner[(~last_election_winner.tipo_eleicao.isna()) & (last_election_winner.tipo_eleicao != 'eleicao ordinaria')].tipo_eleicao).values))

    if (winner4yr.shape[0] != 0) & (winner8yr.shape[0] != 0):
        if (float(winner4yr) == float(winner8yr)):
            final.loc[i, 'term_limited_seat'] = 1

No Turno 1 data (only turno 2) for 3301009 in 2004


In [176]:
this_vote

,ano,turno,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,cargo,sigla_partido,numero_candidato,sequencial_candidato,id_candidato_bd,resultado,votos
13865,2004,2,eleicao ordinaria,RJ,3301009,58190,prefeito,PDT,12,323,963200,eleito,131363
39225,2004,2,eleicao ordinaria,RJ,3301009,58190,prefeito,PMDB,15,319,378227,nao eleito,109309


In [175]:
this_vote[(this_vote.turno == 1) & (this_vote.resultado == '2º turno')]

,ano,turno,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,cargo,sigla_partido,numero_candidato,sequencial_candidato,id_candidato_bd,resultado,votos


In [142]:
final.loc[i,'winner_turno1'] = this_vote.loc[this_vote[(this_vote.turno == 1) & (this_vote.resultado == 'eleito')].votos.idxmax()].id_candidato_bd
final.loc[i,'totalvotes_turno1'] = sum(this_vote[this_vote.turno == 1].votos)
vote_sort1 = sorted(this_vote[this_vote.turno == 1].votos, reverse = True) + [0]
final.loc[i, 'margin_turno1'] = vote_sort1[0] - vote_sort1[1]




In [143]:
last_election_winner

,ano,turno,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,cargo,sigla_partido,numero_candidato,sequencial_candidato,id_candidato_bd,resultado,votos
65004,2012,1,eleicao suplementar guarapari 2012,ES,3202405,56472,prefeito,DEM,25,80000013228,33954,eleito,24709
72800,2012,1,eleicao ordinaria,ES,3202405,56472,prefeito,PSB,40,80000007982,523351,eleito,13846


In [121]:
last_election_winner[~last_election_winner.id_candidato_bd.isna()]

,ano,turno,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,cargo,sigla_partido,numero_candidato,sequencial_candidato,id_candidato_bd,resultado,votos
9639,2012,1,eleicao suplementar camamu-ba,BA,2905800,34150,prefeito,PP,11,50000047946,1621187,eleito,7677


In [ ]:
final.loc[i,'winner_turno1'] = this_vote.loc[this_vote[(this_vote.turno == 1) & (this_vote.resultado == 'eleito')].votos.idxmax()].id_candidato_bd
#final.loc[i,'winner_turno2'] = np.nan

final.loc[i,'totalvotes_turno1'] = sum(this_vote[this_vote.turno == 1].votos)
#final.loc[i,'totalvotes_turno2'] = sum(this_vote[this_vote.turno == 2].votos)

vote_sort1 = sorted(this_vote[this_vote.turno == 1].votos, reverse = True) + [0]
final.loc[i, 'margin_turno1'] = vote_sort1[0] - vote_sort1[1]

if last_election_winner.shape[0] == 1:
    final.loc[i, 'incumbant'] = int((last_election[(last_election.resultado == 'eleito') | (last_election.resultado == 'eleito por qp')].id_candidato_bd).isin(this_vote.id_candidato_bd))

#sometimes are two win options?
if (last_election_winner.shape[0] != 0) & (last_election_winner.shape[0] != 1):
    print(i)

if (winner4yr.shape[0] != 0) & (winner8yr.shape[0] != 0):
    if (float(winner4yr) == float(winner8yr)):
        final.loc[i, 'term_limited_seat'] = 1 

In [112]:
this_vote

,ano,turno,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,cargo,sigla_partido,numero_candidato,sequencial_candidato,id_candidato_bd,resultado,votos
853,2016,1,eleicao ordinaria,BA,2905800,34150,prefeito,REDE,18,50000037473,54482,nao eleito,0
21641,2016,1,eleicao ordinaria,BA,2905800,34150,prefeito,PT,13,50000033312,774143,eleito,8570
22567,2016,1,eleicao suplementar de camamu,BA,2905800,34150,prefeito,PT,13,50000631173,774143,nao eleito,0
57210,2016,1,eleicao suplementar de camamu,BA,2905800,34150,prefeito,CIDADANIA,23,50000631177,1005603,nao eleito,1139
64250,2016,1,eleicao suplementar de camamu,BA,2905800,34150,prefeito,DEM,25,50000631176,89288,nao eleito,6867
71564,2016,1,eleicao ordinaria,BA,2905800,34150,prefeito,PSB,40,50000016883,89288,nao eleito,6551
75312,2016,1,eleicao ordinaria,BA,2905800,34150,prefeito,PV,43,50000032259,345262,nao eleito,1640
88213,2016,1,eleicao suplementar de camamu,BA,2905800,34150,prefeito,PSOL,50,50000631185,61937,nao eleito,340
89090,2016,1,eleicao suplementar de camamu,BA,2905800,34150,prefeito,PATRIOTA,51,50000631171,1492328,eleito,7430


In [164]:
def occupation_label(occ):     
    agriculture = ['tecnico em agronomia e agrimensura', 'trabalhador agricola', 'trabalhador da pecuaria', 
                  'agricultor', 'agronomo', 'aposentado (exceto funcionario publico)',
                   'aposentado (exceto servidor publico)', 'operador de implemento de agricultura pecuaria e exploracao florestal', 
                  'pecuarista', 'produtor agropecuario', 'proprietario de estabelecimento agricola da pecuaria e florestal']

    construction_industrial = ['tecnico de obras civis estradas saneamento e trabalhador assemelhado', 
                    'tecnico em edificacoes', 'torneiro mecanico', 'trabalhador de construcao civil', 
                   'trabalhador metalurgico e siderurgico', 'eletricista e assemelhados', 
                    'encanadorsoldadorchapeadorcaldeireiromontador de estrutura metalic', 'engenheiro', 
                   'industrial', 'mecanico de manutencao', 'operador de aparelhos de producao industrial'
                   'proprietario de estabelecimento de prestacao de servicos']

    politics = ['membros do poder executivo: presidente ministro governador prefeito', 
                'membros do poder legislativo: senador deputados e vereador', 'prefeito',
               'presidente da republica ministro de estado governador e prefeito',
               'senador deputado e vereador']

    business = ['bancario e economiario', 'capitalista de ativos financeiros', 'comerciario', 
                  'corretor de imoveis seguros titulos e valores', 'empresario', 
                  'proprietario de estabelecimento comercial']

    public_employee = ['agente postal', 'assistente social', 'bombeiro civil', 'bombeiro militar',
                      'funcionario publico civil aposentado', 'membro das forcas armadas', 
                       'membro do ministerio publico', 'militar reformado', 'ocupante de cargo em comissao', 
                      'policial civil', 'policial militar', 'serventuario de justica', 'servidor publico civil aposentado', 
                      'servidor publico estadual', 'servidor publico federal', 'servidor publico municipal']

    mining_oil = ["tecnico de mineracao metalurgia e geologia", 'petroleiro']
    
    if occ.isin(agriculture):
        return "agriculture"
    if occ.isin(construction_industrial):
        return "construction_industrial"
    if occ.isin(politics):
        return "politics"
    if occ.isin(business):
        return "business"
    if occ.isin(public_employee):
        return "public_employee"


In [ ]:
demo = df[['id_candidato_bd','idade', 'genero', 'instrucao', 'ocupacao']]
demo.columns = [el + '_turno1' for el in demo.columns]

final = pd.merge(final, demo, how = 'left', left_on = 'winner_turno1', right_on = 'id_candidato_bd_turno1').drop(['id_candidato_bd_turno1'], axis = 1)

demo.columns = [el + '_turno2' for el in demo.columns]
final = pd.merge(final, demo, how = 'left', left_on = 'winner_turno2', right_on = 'id_candidato_bd_turno2').drop(['id_candidato_bd_turno2'], axis = 1)



In [62]:
np.repeat('turno_1', demo.shape[1])

array(['turno_1', 'turno_1', 'turno_1', 'turno_1', 'turno_1'], dtype='<U7')

In [68]:
demo.columns

Index(['id_candidato_bd_turno1', 'idade_turno1', 'genero_turno1',
       'instrucao_turno1', 'ocupacao_turno1'],
      dtype='object')

In [61]:
demo.shape

(7986, 5)

In [69]:
demo = df[['id_candidato_bd_turno1','idade', 'genero', 'instrucao', 'ocupacao']]

KeyError: "['id_candidato_bd_turno1'] not in index"

In [ ]:
final

In [ ]:
last_election_winner.shape[0]

In [ ]:
last_election_winner = last_election[(last_election.resultado == 'eleito') | (last_election.resultado == 'eleito por qp')]



In [ ]:
last_election_winner.shape[0]

In [ ]:
final.head()

In [ ]:
last_election[(last_election.resultado == 'eleito') | (last_election.resultado == 'eleito por qp')]

In [ ]:
int((last_election[(last_election.resultado == 'eleito') | (last_election.resultado == 'eleito por qp')].id_candidato_bd).isin(this_vote.id_candidato_bd))



In [ ]:
last_election[(last_election.resultado == 'eleito') | (last_election.resultado == 'eleito por qp')]

In [22]:
(last_election[last_election.resultado == 'eleito'].id_candidato_bd).isin(this_vote.id_candidato_bd)

Series([], Name: id_candidato_bd, dtype: bool)

In [88]:
float(winner8yr)

nan

In [ ]:
final

In [75]:
winner8yr.shape[0]

0

In [76]:
winner4yr

Series([], Name: id_candidato_bd, dtype: object)

In [65]:
np.array(winner4yr) is None

False

In [62]:
int(winner4yr)

TypeError: cannot convert the series to <class 'int'>

In [59]:
int(winner4yr) == int(winner8yr)

False

In [53]:
winner8yr

80662    543816
Name: id_candidato_bd, dtype: object

In [46]:
last_last_election = vote[(vote.ano == this_year - 8) & (vote.id_municipio == this_mun)]



In [50]:
winner4yr = last_election[last_election.resultado == 'eleito'].id_candidato_bd
winner8yr = last_last_election[last_last_election.resultado == 'eleito'].id_candidato_bd

winner4yr == winner8yr

64939    658310
Name: id_candidato_bd, dtype: object

In [49]:
winner8yr = last_last_election[last_last_election.resultado == 'eleito'].id_candidato_bd


80662    543816
Name: id_candidato_bd, dtype: object

In [25]:
if this_year >= 2000:

SyntaxError: invalid syntax (<ipython-input-25-ddc1edf1cd37>, line 1)

In [23]:
int((last_election[last_election.resultado == 'eleito'].id_candidato_bd).isin(this_vote.id_candidato_bd))

1

In [21]:
this_vote.id_candidato_bd

661      1155584
6441     1391387
13161    1236040
20016     540374
48520    1492751
62263    1144770
Name: id_candidato_bd, dtype: object

In [307]:
(last_election[last_election.resultado == 'eleito'].id_candidato_bd).isin(this_vote.id_candidato_bd)



Series([], Name: id_candidato_bd, dtype: bool)

In [304]:
int((last_election[last_election.resultado == 'eleito'].id_candidato_bd).isin(this_vote.id_candidato_bd))

0

In [293]:
last_election[last_election.resultado == 'eleito'].id_candidato_bd

3490    NaN
Name: id_candidato_bd, dtype: object

In [294]:
sum(np.isin(np.append(np.array(last_election[last_election.resultado == 'eleito'].id_candidato_bd), 0), list(this_vote.id_candidato_bd)))


0

In [271]:
(last_election[last_election.resultado == 'eleito'].id_candidato_bd).isin(list(this_vote.id_candidato_bd))



Series([], Name: id_candidato_bd, dtype: bool)

In [291]:
int(np.isin(np.append(np.array(last_election[last_election.resultado == 'eleito'].id_candidato_bd), 0), list(this_vote.id_candidato_bd)))


TypeError: only size-1 arrays can be converted to Python scalars

In [275]:
(list(last_election[last_election.resultado == 'eleito'].id_candidato_bd) + [0]).isin(list(this_vote.id_candidato_bd))

AttributeError: 'list' object has no attribute 'isin'

In [270]:
list(this_vote.id_candidato_bd)

[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]

In [266]:
last_election

,ano,turno,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,cargo,sigla_partido,numero_candidato,sequencial_candidato,id_candidato_bd,resultado,votos


In [248]:
last_election = vote[(vote.ano == this_year - 4) & (vote.id_municipio == this_mun)]

In [255]:
last_election

,ano,turno,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,cargo,sigla_partido,numero_candidato,sequencial_candidato,id_candidato_bd,resultado,votos
13160,2004,1,eleicao ordinaria,MA,2112209,9377,prefeito,PDT,12,1,1160344,nao eleito,31336
37181,2004,1,eleicao ordinaria,MA,2112209,9377,prefeito,PMDB,15,173,1227829,eleito,34011
94893,2004,1,eleicao ordinaria,MA,2112209,9377,prefeito,PT do B,70,140,1580008,nao eleito,1253


In [259]:
int((last_election[last_election.resultado == 'eleito'].id_candidato_bd).isin(list(this_vote.id_candidato_bd)))

1

In [253]:
list(this_vote.id_candidato_bd)

['1715435', '88246', '1227829', '731442']

In [246]:
vote_sort[0] - vote_sort[1]

7276

In [236]:
sorted(this_vote[this_vote.turno == 1].votos)[-2]

33422

In [203]:
this_vote

,ano,turno,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,cargo,sigla_partido,numero_candidato,sequencial_candidato,id_candidato_bd,resultado,votos
1,1996,1,eleicao ordinaria,AM,1302603,2550,prefeito,PSTU,16,NaN,NaN,nao eleito,1713
3445,1996,1,eleicao ordinaria,AM,1302603,2550,prefeito,PPB,11,NaN,NaN,2º turno,160163
3490,1996,2,eleicao ordinaria,AM,1302603,2550,prefeito,PPB,11,NaN,NaN,eleito,235508
11111,1996,1,eleicao ordinaria,AM,1302603,2550,prefeito,PDT,12,NaN,NaN,nao eleito,44285
31136,1996,1,eleicao ordinaria,AM,1302603,2550,prefeito,PMDB,15,NaN,NaN,nao eleito,125119
66732,1996,1,eleicao ordinaria,AM,1302603,2550,prefeito,PRTB,28,NaN,NaN,nao eleito,367
68072,1996,1,eleicao ordinaria,AM,1302603,2550,prefeito,PMN,33,NaN,NaN,nao eleito,1602
69501,1996,1,eleicao ordinaria,AM,1302603,2550,prefeito,PSB,40,NaN,NaN,2º turno,134475
69512,1996,2,eleicao ordinaria,AM,1302603,2550,prefeito,PSB,40,NaN,NaN,nao eleito,232950
94701,1996,1,eleicao ordinaria,AM,1302603,2550,prefeito,PT do B,70,NaN,NaN,nao eleito,671


In [199]:
this_vote[(this_vote.turno == 2) & (this_vote.resultado == 'eleito')].id_candidato_bd

69511    NaN
Name: id_candidato_bd, dtype: object

In [ ]:
this_vote.loc[this_vote[(this_vote.turno == 1) & (this_vote.resultado == '2º turno')].votos.idxmax()].id_candidato_bd



In [180]:

final.loc[i,'winner_turno1'] = this_vote.loc[this_vote[(this_vote.turno == 1) & (this_vote.resultado == 'eleito')].votos.idxmax()].id_candidato_bd


In [186]:
this_vote

,ano,turno,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,cargo,sigla_partido,numero_candidato,sequencial_candidato,id_candidato_bd,resultado,votos
29,2000,1,eleicao ordinaria,AL,2705002,27995,prefeito,PSD,41,NaN,1267896,nao eleito,2312
11144,2000,1,eleicao ordinaria,AL,2705002,27995,prefeito,PDT,12,NaN,1263076,eleito,6984
25672,2000,1,eleicao ordinaria,AL,2705002,27995,prefeito,PTB,14,NaN,152694,nao eleito,272


In [176]:
final.iloc[99]

ano               2000.0
id_municipio     2308807
runoff               0.0
winner_turno1        NaN
winner_turno2        NaN
Name: 111, dtype: object

In [143]:
this_vote

,ano,turno,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,cargo,sigla_partido,numero_candidato,sequencial_candidato,id_candidato_bd,resultado,votos
1,1996,1,eleicao ordinaria,AM,1302603,2550,prefeito,PSTU,16,NaN,NaN,nao eleito,1713
3445,1996,1,eleicao ordinaria,AM,1302603,2550,prefeito,PPB,11,NaN,NaN,2º turno,160163
3490,1996,2,eleicao ordinaria,AM,1302603,2550,prefeito,PPB,11,NaN,NaN,eleito,235508
11111,1996,1,eleicao ordinaria,AM,1302603,2550,prefeito,PDT,12,NaN,NaN,nao eleito,44285
31136,1996,1,eleicao ordinaria,AM,1302603,2550,prefeito,PMDB,15,NaN,NaN,nao eleito,125119
66732,1996,1,eleicao ordinaria,AM,1302603,2550,prefeito,PRTB,28,NaN,NaN,nao eleito,367
68072,1996,1,eleicao ordinaria,AM,1302603,2550,prefeito,PMN,33,NaN,NaN,nao eleito,1602
69501,1996,1,eleicao ordinaria,AM,1302603,2550,prefeito,PSB,40,NaN,NaN,2º turno,134475
69512,1996,2,eleicao ordinaria,AM,1302603,2550,prefeito,PSB,40,NaN,NaN,nao eleito,232950
94701,1996,1,eleicao ordinaria,AM,1302603,2550,prefeito,PT do B,70,NaN,NaN,nao eleito,671


In [137]:
this_vote.loc[this_vote[(this_vote.turno == 1) & (this_vote.resultado == '2º turno')].votos.idxmax()].id_candidato_bd

nan

In [132]:
this_vote[(this_vote.turno == 1) & (this_vote.resultado == '2º turno')]

,ano,turno,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,cargo,sigla_partido,numero_candidato,sequencial_candidato,id_candidato_bd,resultado,votos
16592,1996,1,eleicao ordinaria,AL,2704302,27855,prefeito,PT,13,NaN,NaN,2º turno,63548
69500,1996,1,eleicao ordinaria,AL,2704302,27855,prefeito,PSB,40,NaN,NaN,2º turno,70271


In [131]:
this_vote[(this_vote.turno == 1) & (this_vote.resultado == '2º turno')].votos.idxmax()

69500

In [130]:
this_vote.iloc[this_vote[(this_vote.turno == 1) & (this_vote.resultado == '2º turno')].votos.idxmax()]

IndexError: single positional indexer is out-of-bounds

In [100]:
this_vote.head()

,ano,turno,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,cargo,sigla_partido,numero_candidato,sequencial_candidato,id_candidato_bd,resultado,votos
0,1996,1,eleicao ordinaria,AL,2704302,27855,prefeito,PSD,41,NaN,NaN,nao eleito,2388
3444,1996,1,eleicao ordinaria,AL,2704302,27855,prefeito,PPB,11,NaN,NaN,nao eleito,3884
16592,1996,1,eleicao ordinaria,AL,2704302,27855,prefeito,PT,13,NaN,NaN,2º turno,63548
16613,1996,2,eleicao ordinaria,AL,2704302,27855,prefeito,PT,13,NaN,NaN,nao eleito,107776
25634,1996,1,eleicao ordinaria,AL,2704302,27855,prefeito,PTB,14,NaN,NaN,nao eleito,41404


In [77]:
unique_combos.iloc[0].ano

1996

In [54]:
keep = df[['ano', 'tipo_eleicao', 'sigla_uf', 'id_municipio', 'id_candidato_bd', 
       'sigla_partido', 'ocupacao', 'cpf',
       'idade', 'genero', 'instrucao']]

In [63]:
vote.head()

,ano,turno,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,cargo,sigla_partido,numero_candidato,sequencial_candidato,id_candidato_bd,resultado,votos
0,1996,1,eleicao ordinaria,AL,2704302,27855,prefeito,PSD,41,NaN,NaN,nao eleito,2388
1,1996,1,eleicao ordinaria,AM,1302603,2550,prefeito,PSTU,16,NaN,NaN,nao eleito,1713
2,1996,1,eleicao ordinaria,BA,2927408,38490,prefeito,PAN,26,NaN,NaN,nao eleito,15721
3,1996,1,eleicao ordinaria,BA,2927408,38490,prefeito,PCO,29,NaN,NaN,nao eleito,705
4,1996,1,eleicao ordinaria,BA,2927408,38490,prefeito,PGT,30,NaN,NaN,nao eleito,989


In [ ]:
for i in range(vote.shape[0]):
    

In [55]:
keep.head()

,ano,tipo_eleicao,sigla_uf,id_municipio,id_candidato_bd,sigla_partido,ocupacao,cpf,idade,genero,instrucao
0,1996,eleicao ordinaria,PI,2211001,NaN,PSTU,NaN,NaN,30,NaN,NaN
1,1996,eleicao ordinaria,SE,2800308,NaN,PSTU,NaN,NaN,40,NaN,NaN
2,1996,eleicao ordinaria,RS,4314902,NaN,PRONA,NaN,NaN,46,NaN,NaN
3,1996,eleicao ordinaria,MG,3106200,NaN,PSTU,NaN,NaN,43,NaN,NaN
4,1996,eleicao ordinaria,PB,2507507,NaN,PSTU,NaN,NaN,43,NaN,NaN


In [59]:
keep['ind'] = np.ones(keep.shape[0])

<ipython-input-59-748716a52b08>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  keep['ind'] = np.ones(keep.shape[0])


In [ ]:
for i in range(keep.shape[0]):
    

In [ ]:
vote

In [61]:
keep[['ano', 'id_municipio', 'ind']].groupby(['ano', 'id_municipio']).sum()

ind
ano  id_municipio     
1996 1200138       3.0
     1200179       3.0
     1200401       8.0
     1200807       5.0
     1302603       8.0
...                ...
2020 5219704       2.0
     5220405       1.0
     5220504       1.0
     5221403       1.0
     5222203       2.0

[6303 rows x 1 columns]

In [44]:
vote[vote.resultado == 'eleito'][['ano', 'id_municipio', 'id_candidato_bd', 'sigla_partido' ,'votos']]



,ano,id_municipio,id_candidato_bd,sigla_partido,votos
34,2000,2708907,1265591,PSD,3147
41,2000,1302553,19147,PST,1455
49,2000,2903300,1071793,PAN,2135
51,2000,2906907,962907,PSD,4614
54,2000,2908606,1523132,PST,3643
...,...,...,...,...,...
96803,2020,1101807,932364,PROS,2484
96813,2020,3502754,1706236,PROS,5124
96830,2020,1704105,1659020,PROS,635
96831,2020,1709500,1719990,PROS,20116


In [42]:
vote[vote.turno==2]

,ano,turno,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,cargo,sigla_partido,numero_candidato,sequencial_candidato,id_candidato_bd,resultado,votos
527,2000,2,eleicao ordinaria,PA,1501402,4278,prefeito,PSD,41,NaN,1632929,nao eleito,316279
1215,2016,2,eleicao ordinaria,AP,1600303,6050,prefeito,REDE,18,30000002139,31551,eleito,123808
1216,2016,2,eleicao ordinaria,CE,2303709,13730,prefeito,PMB,35,60000011594,140066,eleito,80756
1217,2016,2,eleicao ordinaria,ES,3205002,56995,prefeito,REDE,18,80000011517,688134,eleito,112344
1218,2016,2,eleicao ordinaria,PR,4119905,77771,prefeito,REDE,18,160000011560,1177454,nao eleito,79008
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95847,2016,2,eleicao ordinaria,PE,2609600,24910,prefeito,SOLIDARIEDADE,77,170000020870,497950,eleito,120225
96324,2020,2,eleicao ordinaria,MG,3170107,54011,prefeito,SOLIDARIEDADE,77,130001037220,1104943,eleito,85990
96325,2020,2,eleicao ordinaria,RR,1400100,3018,prefeito,SOLIDARIEDADE,77,230000840534,466039,nao eleito,20032
96836,2020,2,eleicao ordinaria,CE,2303709,13730,prefeito,PROS,90,60000718891,441082,eleito,83588


In [25]:
vote.head()

,ano,turno,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,cargo,sigla_partido,numero_candidato,sequencial_candidato,id_candidato_bd,resultado,votos
0,1996,1,eleicao ordinaria,AL,2704302,27855,prefeito,PSD,41,NaN,NaN,nao eleito,2388
1,1996,1,eleicao ordinaria,AM,1302603,2550,prefeito,PSTU,16,NaN,NaN,nao eleito,1713
2,1996,1,eleicao ordinaria,BA,2927408,38490,prefeito,PAN,26,NaN,NaN,nao eleito,15721
3,1996,1,eleicao ordinaria,BA,2927408,38490,prefeito,PCO,29,NaN,NaN,nao eleito,705
4,1996,1,eleicao ordinaria,BA,2927408,38490,prefeito,PGT,30,NaN,NaN,nao eleito,989


In [5]:
#for each mun and each year want winning candidate, bio info, margin etc, keep cpf

#votes, total votes, pct votes, political party, incumbent runs, incumbent wins, seat limit


#for occupation don't make dummies, agro and political indicators only 
# categories, politician, agriculture, mining

#raca, maybe skip since missing for early years
#sigla partido also make categorical, PT, PSDB, PTB, [MDB, PMDB](same party diff names) 




In [6]:
vote.head()

,ano,turno,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,cargo,sigla_partido,numero_candidato,sequencial_candidato,id_candidato_bd,resultado,votos
0,1996,1,eleicao ordinaria,AL,2704302,27855,prefeito,PSD,41,NaN,NaN,nao eleito,2388
1,1996,1,eleicao ordinaria,AM,1302603,2550,prefeito,PSTU,16,NaN,NaN,nao eleito,1713
2,1996,1,eleicao ordinaria,BA,2927408,38490,prefeito,PAN,26,NaN,NaN,nao eleito,15721
3,1996,1,eleicao ordinaria,BA,2927408,38490,prefeito,PCO,29,NaN,NaN,nao eleito,705
4,1996,1,eleicao ordinaria,BA,2927408,38490,prefeito,PGT,30,NaN,NaN,nao eleito,989


In [7]:
np.array(vote.columns)

array(['ano', 'turno', 'tipo_eleicao', 'sigla_uf', 'id_municipio',
       'id_municipio_tse', 'cargo', 'sigla_partido', 'numero_candidato',
       'sequencial_candidato', 'id_candidato_bd', 'resultado', 'votos'],
      dtype=object)

In [5]:
keep = vote[vote.resultado == 'eleito'][['ano', 'id_municipio', 'id_candidato_bd', 'sigla_partido' ,'votos']]
keep.columns = ['ano', 'id_municipio', 'id_candidato_bd', 'sigla_partido', 'votos_winner']

#total votes for that municipio that year
keep = pd.merge(keep, vote[['ano', 'id_municipio', 'votos']].groupby(['ano', 'id_municipio']).agg('sum').reset_index(), how = 'left', left_on = ['ano', 'id_municipio'], right_on = ['ano', 'id_municipio'])
keep.head()

,ano,id_municipio,id_candidato_bd,sigla_partido,votos_winner,votos
0,2000,2708907,1265591,PSD,3147,4882
1,2000,1302553,19147,PST,1455,5658
2,2000,2903300,1071793,PAN,2135,3598
3,2000,2906907,962907,PSD,4614,8760
4,2000,2908606,1523132,PST,3643,8872


In [6]:
keep['incumbant_run'] = np.zeros(keep.shape[0])
keep['incumbant_win'] = np.zeros(keep.shape[0])
keep['term_lim_seat'] = np.zeros(keep.shape[0])
keep['margin'] = np.zeros(keep.shape[0])

In [7]:
#first find does incumbant run and does incumbant win? 
missing_incumbant = []

for i in tqdm(range(vote.shape[0])):
    row = vote.iloc[i]
    row

    #did they win? 
    if row.resultado == 'eleito':
        #check 4 years from current year for year+4 and id match
        incumbant_run = vote[(vote.ano == row.ano + 4) & (vote.id_candidato_bd == row.id_candidato_bd) & (vote.id_municipio == row.id_municipio)].shape[0]
        
       
        #if they run in 4 years, check if they win 
        if incumbant_run == 1:
            if vote[(vote.ano == row.ano + 4) & (vote.id_candidato_bd == row.id_candidato_bd)].shape[0] == 0:
                print('Missing data with :' + str(i))

            else:
                incumbant_win = int(vote[(vote.ano == row.ano + 4) & (vote.id_candidato_bd == row.id_candidato_bd)].resultado == 'eleito')
        
        
        #if they do not run in 4 years, win is 0
        if incumbant_run==0:
            incumbant_win = 0
        
        #stop if year is 2020 because 2024 not in dataset
        if row.ano == 2020:
            continue
        
        #update incumbent run and incumbent win for each row
        if keep[(keep.ano == row.ano+4)&(keep.id_municipio == row.id_municipio)].shape[0] == 0:
            #print('Missing row at :' + str(i) + 'year = ' + str(row.ano+4) + ' munic = ' + str(row.id_municipio))
            missing_incumbant.append([i, row.ano+4,row.id_municipio])
        else:
            keep.loc[keep[(keep.ano == row.ano+4)&(keep.id_municipio == row.id_municipio)].index[0], 'incumbant_run'] = incumbant_run
            keep.loc[keep[(keep.ano == row.ano+4)&(keep.id_municipio == row.id_municipio)].index[0], 'incumbant_win'] = incumbant_win




100%|██████████| 96838/96838 [14:21<00:00, 112.37it/s] 


In [8]:
keep['term_lim_seat'] = np.zeros(keep.shape[0])
missing_lim = []

for i in tqdm(range(vote.shape[0])):
    row = vote.iloc[i]
    
    #first check this person won
    if row.resultado == 'eleito':
        
        #check that same person won in same muni in 4 years
        if vote[(vote.id_municipio == row.id_municipio) & (vote.resultado == 'eleito') & (vote.id_candidato_bd == row.id_candidato_bd) & (vote.ano == row.ano + 4)].shape[0] != 0:
            
            if keep[(keep.ano == row.ano + 8) & (keep.id_municipio == row.id_municipio)].shape[0] == 0:
                missing_lim.append((row.ano, row.id_municipio, row.id_candidato_bd))
           
            elif keep[(keep.ano == row.ano + 8) & (keep.id_municipio == row.id_municipio)].shape[0] == 1:
                keep.loc[keep[(keep.ano == row.ano + 8) & (keep.id_municipio == row.id_municipio)].index[0], 'term_lim_seat'] = 1



        
    

100%|██████████| 96838/96838 [11:46<00:00, 137.15it/s] 


In [9]:
# #now do if the seat is term limited: 
# missing = []

# for i in tqdm(range(vote.shape[0])):
#     row = vote.iloc[i]
    
#     temp_frame = vote[(vote.id_candidato_bd == row.id_candidato_bd)&(vote.resultado == 'eleito')]
#     temp_frame = temp_frame.sort_values('ano').reset_index()
    
#     for index, row2 in temp_frame.iterrows():
#         if temp_frame[temp_frame.ano == row2.ano+4].shape[0] != 0:
            
#             if keep[(keep.ano == row2.ano + 8) & (keep.id_municipio == row2.id_municipio)].shape[0] == 0:
#                 missing.append([i,row2.ano+8,row2.id_municipio])
#                 #missing.append('Missing at ' + str(i) + ' year = ' + str(row.ano) + str(' munic = ') + str(row.id_municipio))
                                 
#             else:
#                 keep.loc[keep[(keep.ano == row2.ano + 8) & (keep.id_municipio == row2.id_municipio)].index[0], 'term_lim_seat'] = 1


    
    

In [10]:
#now get margin of victory


vote_missing = []
for i in tqdm(range(keep.shape[0])):
    row = keep.iloc[i]
    
    if vote[(vote.ano == row.ano) & (vote.id_municipio == row.id_municipio)].shape[0] == 0:
        vote_missing.append(i, row.ano, row.id_municipio)
    else:
        vote_sort = sorted(vote[(vote.ano == row.ano) & (vote.id_municipio == row.id_municipio)].votos, reverse = True)
        vote_sort.append(0)
        margin = vote_sort[0] - vote_sort[1]
        
    if keep[(keep.ano == row.ano) & (keep.id_municipio == row.id_municipio)].shape[0] != 0:
        keep.loc[keep[(keep.ano == row.ano) & (keep.id_municipio == row.id_municipio)].index[0], 'margin'] = margin
        
        
        

100%|██████████| 33756/33756 [11:05<00:00, 50.70it/s]


In [11]:
keep['votos_pct'] = keep.votos_winner/keep.votos

In [16]:
keep2 = pd.merge(keep, df[['id_candidato_bd', 'genero', 'ocupacao', 'instrucao', 'ano', 'idade']], how = 'left', left_on = ['id_candidato_bd', 'ano'], right_on = ['id_candidato_bd', 'ano'])




In [17]:
keep2.head()

,ano,id_municipio,id_candidato_bd,sigla_partido,votos_winner,votos,incumbant_run,incumbant_win,term_lim_seat,margin,votos_pct,genero,ocupacao,instrucao,idade
0,2000,2708907,1265591,PSD,3147,4882,0.0,0.0,0.0,1412.0,0.644613,masculino,proprietario de estabelecimento comercial,ensino medio completo,45
1,2000,1302553,19147,PST,1455,5658,0.0,0.0,0.0,155.0,0.257158,masculino,medico,ensino superior completo,31
2,2000,2903300,1071793,PAN,2135,3598,0.0,0.0,0.0,708.0,0.593385,masculino,outros,ensino superior completo,51
3,2000,2906907,962907,PSD,4614,8760,0.0,0.0,0.0,640.0,0.526712,masculino,outros,ensino fundamental incompleto,58
4,2000,2908606,1523132,PST,3643,8872,0.0,0.0,0.0,1015.0,0.410618,masculino,proprietario de estabelecimento comercial,ensino fundamental completo,42


In [18]:
keep2.shape

(125491, 15)

In [96]:
#investigate where people ran in different munis for kathryn

# not_same = []

# for i in tqdm(range(vote.shape[0])):
#     row = vote.iloc[i]
#     if row.resultado == 'eleito':
#         if len((np.unique(vote[(vote.id_candidato_bd == row.id_candidato_bd)].id_municipio))) != 1:
#             not_same.append([row.id_candidato_bd, np.unique(vote[(vote.id_candidato_bd == row.id_candidato_bd)].id_municipio)])
    

# not_same_df = pd.DataFrame(not_same, columns = ['id_candidato_bd', 'munis'])
# not_same_df = not_same_df[not_same_df['id_candidato_bd'].notna()]
# not_same_df = not_same_df.drop_duplicates(subset='id_candidato_bd', keep="first")
# not_same_df.to_csv('different_munis_election.csv')

100%|██████████| 96838/96838 [03:31<00:00, 457.34it/s] 


In [284]:
#pd.merge(keep, vote[['ano', 'id_municipio', 'votos']].groupby(['ano', 'id_municipio']).agg('sum').reset_index(), how = 'left', left_on = ['ano', 'id_municipio'], right_on = ['ano', 'id_municipio'])



In [230]:
keep2 = pd.merge(keep, df[['id_candidato_bd', 'genero', 'ocupacao', 'instrucao']], how = 'left', left_on = ['id_candidato_bd', 'ano'], right_on = ['id_candidato_bd', 'ano'])


In [231]:
keep.to_csv('election_munic.csv')

In [ ]:
#

In [237]:
missing_df = pd.DataFrame(missing_incumbant, columns = ['index', 'ano', 'id_candidato_bd'])

In [238]:
votes_missing_df = pd.DataFrame(missing, columns = ['index', 'ano', 'id_candidato_bd'])

In [244]:
keep.head()

,ano,id_municipio,id_candidato_bd,sigla_partido,votos_winner,votos,incumbant_run,incumbant_win,term_lim_seat,margin,votos_pct,genero,ocupacao,instrucao
0,2000,2708907,1265591,PSD,3147,4882,0.0,0.0,0.0,1412.0,0.644613,masculino,proprietario de estabelecimento comercial,ensino medio completo
1,2000,1302553,19147,PST,1455,5658,0.0,0.0,0.0,155.0,0.257158,masculino,medico,ensino superior completo
2,2000,1302553,19147,PST,1455,5658,0.0,0.0,0.0,155.0,0.257158,masculino,medico,ensino superior completo
3,2000,2903300,1071793,PAN,2135,3598,0.0,0.0,0.0,708.0,0.593385,masculino,outros,ensino superior completo
4,2000,2903300,1071793,PAN,2135,3598,0.0,0.0,0.0,708.0,0.593385,masculino,aposentado (exceto servidor publico),ensino superior completo


In [243]:
vote[]

,ano,turno,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,cargo,sigla_partido,numero_candidato,sequencial_candidato,id_candidato_bd,resultado,votos
0,1996,1,eleicao ordinaria,AL,2704302,27855,prefeito,PSD,41,NaN,NaN,nao eleito,2388
1,1996,1,eleicao ordinaria,AM,1302603,2550,prefeito,PSTU,16,NaN,NaN,nao eleito,1713
2,1996,1,eleicao ordinaria,BA,2927408,38490,prefeito,PAN,26,NaN,NaN,nao eleito,15721
3,1996,1,eleicao ordinaria,BA,2927408,38490,prefeito,PCO,29,NaN,NaN,nao eleito,705
4,1996,1,eleicao ordinaria,BA,2927408,38490,prefeito,PGT,30,NaN,NaN,nao eleito,989


In [246]:
vote_missing

[]